In [1]:
# Se descarga en el directorio actual el documento 'minsearch.py' de la ruta indicada
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-04 19:06:56--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-04 19:06:56 (47.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch

In [2]:
# se importa documents.json, contiene documentos por cada curso(data-engieering, ml, etc) que incluye text, section y question.
import json

with open('documents.json') as documentos_curso:
    documents = json.load(documentos_curso)

In [3]:
print(f'Tamaño: {len(documents)}')
print(f'Tipo: {type(documents)}\n')

for i in range(len(documents)):
    print(documents[i]['course'] )


Tamaño: 3
Tipo: <class 'list'>

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [4]:
# primer elemento de documents del curso 'data-engineering-zoomcamp'
documents[0]['documents'][0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

In [5]:
# Estructura
'''
Ejemplo del primer elemento de document. (Contiene 3) 
>> documents[0]
'course': string 
        'data-engineering-zoomcamp'
'documents': list of dictionaries 
        [ {text: '', section: '' , question: ''},
          {text: '', section: '' , question: ''},
                         ... 
          {text: '', section: '' , question: ''} ]

'''
# Lista para almacenar el conjunto total de documents. 
# Tipo: Lista de diccionarios
# Se va a utilizar para indexar
docs=[]

for i in documents:
    for elemento in i['documents']:
        elemento['course'] = i['course'] # agregamos esta columna
        docs.append(elemento) 
    

In [6]:
docs[15]
# documentos para indexar

{'text': 'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]',
 'section': 'General course-related questions',
 'question': 'Homework - Are late submissions of homework allowed?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
'''
Usamos de minsearch.py la clase Index para crear el objeto 'index'.
Se inicializa con los atributos de text_fields, keyword_fields
Luego aplicamos la función fit con los documentos preparados (docs)

NOTA: revisar a mayor profundidad el documento 'minsearch.py'
'''

"\nUsamos de minsearch.py la clase Index para crear el objeto 'index'.\nSe inicializa con los atributos de text_fields, keyword_fields\n"

In [7]:
# indexamos los documentos con minsearch
index= minsearch.Index(
    text_fields =['question','text','section'],  #for performance search
    keyword_fields=['course'],
        
)

index.fit(docs)

In [ ]:
'''
Función search, retorna lista de diccionarios (documentos) donde cada elemento coincide con el criterio de búsqueda.
Parámetros dados: query, diccionario de pesos, diccionario de filtros, número de resultados 
'''

In [8]:
#query
pregunta= 'the course has already started. Can I still joint it?'
pesos_diccionario= {'text':0.2, 'section':0.5,'question':4.0}
curso_filtro = {'course':'data-engineering-zoomcamp'}

resultados= index.search(
    query= pregunta ,
    boost_dict= pesos_diccionario,
    filter_dict= curso_filtro, 
    num_results= 5
)

resultados


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [9]:
print(type(resultados))

<class 'list'>


In [15]:
'''
Se necesita construir un formato de CONTEXT para ser agregado a prompt. 
Se ordena los elementos de 'resultados'.

Output esperado (string):
"{section:'', question: '', text:'' } ,... , {} "
'''
contexto= ""
for i in resultados:
    contexto = contexto + f"Section: {i['section']} \nQuestion: {i['question']} \nText: {i['text']} \n\n" 

# resultados[0]['text']
print(contexto)


Section: General course-related questions 
Question: Course - Can I still join the course after the start date? 
Text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

Section: General course-related questions 
Question: Course - Can I follow the course after it finishes? 
Text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 

Section: General course-related questions 
Question: Course - What can I do before the course starts? 
Text: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terra

In [16]:
# q
pregunta

'the course has already started. Can I still joint it?'

In [ ]:
'''
Juntamos todo en el prompt y formateamos con la query(pregunta) y la context(base de datos)
'''

In [29]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION from FAQ 
database.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context} 

"""
# strip: elimina espacios en blanco, al inicio y al final
prompt_final= prompt_template.format(question= pregunta, context= contexto).strip() 


In [30]:
print(prompt_final)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION from FAQ 
database.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started. Can I still joint it?

CONTEXT: 
Section: General course-related questions 
Question: Course - Can I still join the course after the start date? 
Text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

Section: General course-related questions 
Question: Course - Can I follow the course after it finishes? 
Text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your 

In [ ]:
'''
 Insertamos el prompt en nuestro modelo LLM
'''

In [3]:
import os
api_key= os.environ.get('OPENAI_API_KEY') # verifica que el api_key exista

In [7]:
import openai
from openai import OpenAI

client = OpenAI()

In [8]:
client

In [10]:
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages = [
        {
            "role":"user",
            "content": pregunta
        }
    ]
)

In [26]:
# respuesta sin contexto 
response.choices[0].message.content

'It depends on the policies of the institution or organization offering the course. Many courses allow late enrollments, while others may have strict deadlines. I recommend reaching out to the course instructor or administration to inquire about the possibility of joining the course, even if it has already started. They can provide you with the best information regarding availability and any necessary steps you need to take.'

In [ ]:
#RESPUESTA CON EL PROMPT CON EL CONTEXTO 

In [36]:
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages = [
        {
            "role":"user",
            "content": prompt_final
        }
    ]
)

In [37]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [41]:
# respuesta en español
prompt_final=prompt_template.format(question='El curso ya comenzó, puedo unirme todavía?', context= contexto).strip()

response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages = [
        {
            "role":"user",
            "content": prompt_final
        }
    ]
)

response.choices[0].message.content

'Sí, puedes unirte al curso aunque ya haya comenzado. Aún eres elegible para enviar las tareas, pero ten en cuenta que habrá fechas límite para entregar los proyectos finales.'

In [ ]:
'''
A continuación:
Modularización: https://www.youtube.com/watch?v=vkTiVwwch6A&list=PL3MmuxUbc_hIB4fSqLy_0AfTjVLpgjV3R&index=7&ab_channel=DataTalksClub%E2%AC%9B
'''